In [1]:
"""
   This notebook computes a median burn date for a given fire polygon from DBCA060. 
   For each fire poylgon, we search the MODIS ESA_CCI_FireCCI_5_1 250m dataset which provides
   daily burnt area estimates from 2001 - 2019. 
   We use folium and geemap to visualize fires and export a csv with the following columns:
       fire_ID, original_date, and MODIS_median_date. 
       
   With questions send me an email: 1dandixon@gmail.com

"""

import ee
ee.Initialize()
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from shapely.geometry import Point,Polygon
import json
import geemap

# to visualize burn area (BA) julien dates of fire from MODIS at 250 m resolution
baVis = {
  'min': 1,
  'max': 366,
  'palette' : [
                'ff0000', 'fd4100', 'fb8200', 'f9c400', 'f2ff00', 'b6ff05',
                '7aff0a', '3eff0f', '02ff15', '00ff55', '00ff99', '00ffdd',
                '00ddff', '0098ff', '0052ff', '0210ff', '3a0dfb', '7209f6',
                'a905f1', 'e102ed', 'ff00cc', 'ff0089', 'ff0047', 'ff0004'
  ]
}

In [2]:
# Preprocess

# All DBCA060 from 2000 to 2020 intersecting NJF, SJF and WARREN
data = gpd.read_file("MODIS_date_check/data/DBCA060_2000-2020_NJF_SJF_WAR_50s.shp")
# generate a single polygon for each record (multipart to single part geometry)
# this step is essential to compute each polygon independenlty instead of records with multi polygons
data = data.explode().reset_index() 

# create a unique ID for each record
data['ID'] = [str(i) for i in range(0, len(data))]
data['ID'] = data['ID'].apply(lambda x: x.zfill(5)) + "_ID"

print(data.head())


   level_0  level_1 fih_fire_s  fih_year1 fih_season fih_distri fih_hist_d  \
0        0        0  2006/2007       2006         ND        PHS       None   
1        1        0  2006/2007       2006         ND        PHS       None   
2        2        0  2013/2014       2013         ND        PHS       None   
3        3        0  2007/2008       2007         ND        WTN       None   
4        4        0  2007/2008       2007         ND        PHS       None   

  fih_number fih_fire_t   fih_date1  ...            fih_commen fih_name  \
0        999         PL  2006-07-01  ...  FPC Plantations data     None   
1        999         PL  2006-07-01  ...  FPC Plantations data     None   
2        999         PL  2013-07-01  ...  FPC Plantations data     None   
3        999         PL  2007-07-01  ...  FPC Plantations data     None   
4        999         PL  2007-07-01  ...  FPC Plantations data     None   

   fih_burn_p fih_master fih_perime fih_hectar   Shape_Leng     Shape_Area  \
0 

In [3]:
# subset data based on area and convert to hectares
data['area2'] = data['geometry'].area * 0.0001

# select records over 50 ha
data = data[data.area2 > 50]

#data.to_file("MODIS_date_check/data/DBCA060_2000-2020_NJF_SJF_WAR_50s_gte50ha.shp") # export the data if required
print("There are {} records".format(len(data)))
print("{} ha is the largest".format(np.round(data.area2.max(), 1)))


There are 2708 records
97846.1 ha is the largest


In [11]:
# Subset the fires by year and plot the fires of interest on a map with ID available 
df = data.copy()
df = df.to_crs('EPSG:4326') # convert to 4326 to work with json and create ee features

df = df[df.fih_year1 == 2018] # for example to see all fires from 2019
m = folium.Map(location=[-31.5, 116.15], zoom_start=9)
for _, r in df.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['ID']).add_to(geo_j)
    geo_j.add_to(m)
m
#print(df.ID.unique())

In [10]:
# first, a set of functions that inputs a fire polygon ID and returns the median MODIS burn area date
# pick a fire from the map above by clicking on a fire and check how it works

ID = '09316_ID' 

row = df[df.ID == ID]
polygon = ee.FeatureCollection(json.loads(gpd.GeoSeries(row['geometry']).simplify(tolerance=0.001).to_json())) # simplify geometry

# the MODIS 250 m dataset #https://developers.google.com/earth-engine/datasets/catalog/ESA_CCI_FireCCI_5_1
MODIS_250 = ee.ImageCollection("ESA/CCI/FireCCI/5_1")

# to create a centroid of the polygon for map centering
def getXY(pt):
    return (pt.x, pt.y)

x, y = getXY(row['geometry'].iloc[0].centroid)

# call geemap for plotting 
Map = geemap.Map(center=(y, x), zoom = 11, basemap = 'SATELLITE')
# add the fire polygon of interest
Map.addLayer(polygon)

# for now use a 380 day window before and after the DBCA fire date
fire_poly = ee.Feature(ee.FeatureCollection(polygon).first()) # don't want anything outside the fire
fire_date = ee.Date(row['fih_date1'].iloc[0]) # fetch the dbca fire date
fire_date_pre = fire_date.advance(-380, 'day') # search all modis before fire
fire_date_post = fire_date.advance(380, 'day') # search all modsi after fire

# clip modis to the dbca fire boundary
def clipIm(image):
    return image.clip(fire_poly)

# define the image collection and filter by date range and clip
MODISfire = ee.ImageCollection('ESA/CCI/FireCCI/5_1')\
                  .filterDate(fire_date_pre, fire_date_post)\
                  .map(clipIm)

# assign ms date info to do median calculation
def assignMS(image):
    year = ee.String(ee.String(image.get('system:index')).split("_").get(0))
    first_of_year = ee.Date(year.cat('-01-01')).millis()
    first_of_year_band = ee.Image(ee.Number(first_of_year)).rename('ms_start').int64()
    # one day = 86400000 milliseconds
    doyMillis = image.select('BurnDate').multiply(ee.Number(86400000)).subtract(86400000)#// to account for the additional day on jan 1
    newDateMillis = first_of_year_band.add(doyMillis).rename('newDateMillis')
    return image.addBands(newDateMillis)

# assign the ms date info    
MODISfire = MODISfire.map(assignMS)
# get the min date value for that modis pixel
burnedArea = MODISfire.select('newDateMillis').reduce(ee.Reducer.min())
# then take the median of all min ms date pixels occuring within the dbca polygon
burnDate = burnedArea.reduceRegion(
              reducer = ee.Reducer.median(),
              geometry = fire_poly.geometry(),
              scale = 250) 

result = burnDate.getInfo() # bring the result to the client side
if not result['newDateMillis_min']:
    # if there's no modis fire within the dbca polygon, use the original dbca date
    original_burn_date = ee.Date(fire_date).format('YYYY-MM-dd')
    print("DBCA fire date: {}".format(fire_date.format('YYYY-MM-dd').getInfo()))
    print("No MODIS detected")

else:
    # a modis fire was detected within the polygon during that time frame
    modis_burn_date = ee.Date(burnDate.get('newDateMillis_min')).format('YYYY-MM-dd')
    print("DBCA fire date: {}".format(fire_date.format('YYYY-MM-dd').getInfo()))
    print("MODIS medain fire date: {}".format(modis_burn_date.getInfo()))

ba = MODISfire.select('BurnDate').min()
Map.addLayer(ba, baVis, 'ba')    

Map


DBCA fire date: 2019-12-14
MODIS medain fire date: 2019-12-20


Map(center=[-33.16621773405393, 116.26195746964383], controls=(WidgetControl(options=['position', 'transparent…

In [6]:

# the same function but suited for server side and map over the whole feature collection of dbca 060
def get_modis_dates(polygon):

    fire_poly = ee.Feature(polygon)#.buffer(-250)# don't want anything outside the fire    
    fire_date = ee.Date(polygon.get('fih_date1'))
    # pre and post date ranges to search modis imagery
    fire_date_pre = fire_date.advance(-380, 'day')
    fire_date_post = fire_date.advance(380, 'day')

    # clip modis to the dbca fire boundary
    def clipIm(image):
        return image.clip(fire_poly)

    # define the image collection and filter by date range and clip
    MODISfire = ee.ImageCollection('ESA/CCI/FireCCI/5_1')\
                      .filterDate(fire_date_pre, fire_date_post)\
                      .map(clipIm)\
    
    # have to assign each pixel in the MODIS image a millisecond date since the epoch
    # that way we can do median calculations on all pixels within the fire polygon that wouldn't be possible
    # using julien dates
    def assignMS(image):
        year = ee.String(ee.String(image.get('system:index')).split("_").get(0))
        first_of_year = ee.Date(year.cat('-01-01')).millis()
        first_of_year_band = ee.Image(ee.Number(first_of_year)).rename('ms_start').int64()
        # one day = 86400000 milliseconds
        doyMillis = image.select('BurnDate').multiply(ee.Number(86400000)).subtract(86400000)#// to account for the additional day on jan 1
        newDateMillis = first_of_year_band.add(doyMillis).rename('newDateMillis')
        return image.addBands(newDateMillis)

    # assign the ms info    
    MODISfire = MODISfire.map(assignMS)
    # get the mean date value for that modis pixel
    burnedArea = MODISfire.select('newDateMillis').reduce(ee.Reducer.mean())
    # then take the median of all mean pixels occuring within the dbca polygon
    burnDate = burnedArea.reduceRegion(
                  reducer = ee.Reducer.median(),
                  geometry = fire_poly.geometry(),
                  scale = 250) 
    # calling .getinfo to see if there was actually a fire encountered 
    #result = burnDate.getInfo()
    polygon = polygon.set('modis_date', burnDate)#.get('newDateMillis_mean'))
    return polygon
    
# for each year and for each polygon, calculate and return modis median fire date along with original date
list_ = []
#for year in range(2001, 2019 + 1): # for all fires 2001 - 2019 (2020 is not included in the dataset yet)
for year in [2018]:# just run for 2018     
    print(year)
    # this is dbca060 uploaded to earth engine as feature collection - i can give you acces to this or you can upload yourself
    polygons = ee.FeatureCollection('users/danieljdixon1991/SWWA/Fires/DBCA060_2000-2020_NJF_SJF_WAR_50s_gte50ha')

    polygons = polygons.filterMetadata('fih_year1','equals', year)

    results = polygons.map(get_modis_dates)

    l = []
    for i in results.getInfo()['features']:
        modis_date = i['properties']['modis_date']['newDateMillis_mean']
        original_date = i['properties']['fih_date1']
        ID = i['properties']['ID']
        l.append((ID, original_date, modis_date))

    output = pd.DataFrame(l, columns = ['ID', 'original_date', 'modis_date'])
    #print(output)
    list_.append(output)


2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [7]:
# Convert the ms dates to format YYYY-MM-dd and convert to pandas datetime
df2 = pd.concat(list_)
df2['modis_date'] = pd.to_datetime(df2['modis_date'], unit='ms')
df2['original_date'] = pd.to_datetime(df2['original_date'])

print(df2)

#df2.to_csv("MODIS_date_check/data/shift_results_2001-2019_NJF_SJF_WAR.csv", index = False)

           ID original_date modis_date
0    05247_ID    2001-07-01        NaT
1    05418_ID    2001-07-01 2002-05-27
2    00408_ID    2001-09-16        NaT
3    07272_ID    2001-01-01 2002-01-20
4    07284_ID    2001-01-01        NaT
..        ...           ...        ...
121  08741_ID    2019-10-25 2019-10-27
122  08742_ID    2019-10-25 2019-10-22
123  08743_ID    2019-10-25        NaT
124  08747_ID    2019-11-28        NaT
125  08381_ID    2019-05-10        NaT

[2464 rows x 3 columns]


In [8]:
n_fire_count = len(df2)
modis_fires = len(df2[~df2.modis_date.isnull()])

print("found a modis date for {} out of {} fires".format(modis_fires, n_fire_count))


found a modis date for 1412 out of 2464 fires
